In [9]:
import xarray as xr
import numpy as np

In [16]:
test_file = '/mnt/nvme2tb/aggelos_test_clouds/modis/geoprocessed/20100101091500_aqua.nc'

In [17]:
ds = xr.open_dataset(test_file)

In [38]:
ds.latitude.values.min(), ds.latitude.values.max(), ds.longitude.values.min(), ds.longitude.values.max()

(23.356104, 44.622437, 43.106773, 73.3539)

In [90]:
resolution = 0.01
lat_bnds = ds.latitude.values.min(), ds.latitude.values.max()
lon_bnds = ds.longitude.values.min(), ds.longitude.values.max()
latitudes = np.arange(lat_bnds[0], lat_bnds[1]+resolution, resolution)
longitudes = np.arange(lon_bnds[0], lon_bnds[1]+resolution, resolution)
grid =  np.meshgrid(latitudes, longitudes)

In [91]:
grid[0].shape, grid[1].shape

((3026, 2128), (3026, 2128))

In [92]:
lats = grid[0][0]

In [93]:
longs = np.array([i[0] for i in grid[1]])

In [94]:
len(lats), len(longs)

(2128, 3026)

In [99]:
ds.Rad[0]

<xarray.DataArray 'Rad' (y: 2030, x: 1354)> Size: 11MB
[2748620 values with dtype=float32]
Coordinates:
    band        <U4 16B '1'
    cloud_mask  (y, x) float32 11MB ...
Dimensions without coordinates: y, x
Attributes:
    grid_mapping:  spatial_ref

In [96]:
ds = ds.drop_vars(['latitude', 'longitude'])

In [86]:
#ds = ds.drop_dims(['latitude', 'longitude'])
ds = ds.expand_dims({'latitude': lats, 'longitude': longs})

## HDF

In [1]:
import os
import xarray as xr
import numpy as np
import pandas as pd
import glob
from glob import glob
from datetime import datetime

import satpy
from satpy.scene import Scene
from satpy import find_files_and_readers
import pyresample as prs

# Python libraries for visualization
import matplotlib.colors
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.geoaxes import GeoAxes
GeoAxes._pcolormesh_patched = Axes.pcolormesh
import geotiepoints
from geotiepoints.geointerpolator import GeoInterpolator

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action = "ignore", category = RuntimeWarning)
warnings.simplefilter(action = "ignore", category = UserWarning)

In [3]:
file_names = ['/mnt/nvme2tb/aggelos_test_clouds/modis/L1B/MYD021KM.A2010001.1200.061.2018053180554.hdf']

In [9]:
import rioxarray
rds = rioxarray.open_rasterio(
    file_names[0],
    variable="EV_1KM_RefSB"
)

In [18]:
nx = len(rds.x)
ny = len(rds.y)

In [20]:
south = 44.261604353775
north = 66.5240751232288
west = 0.226265723803612
east = 46.1296399569942

In [29]:
print(rds.EV_1KM_RefSB.values.shape) 

(15, 2030, 1354)


In [30]:
data_values = rds.EV_1KM_RefSB.values

In [32]:
bands = np.arange(1, data_values.shape[0] + 1)

# Generate 1D arrays of latitudes and longitudes
latitudes = np.linspace(north, south, nx)  # North to South for latitude
longitudes = np.linspace(west, east, ny)  # West to East for longitude

dataset = xr.Dataset(
    {
        "reflectance": (["band", "longitude", "latitude"], data_values),
    },
    coords={
        "band": bands,
        "latitude": latitudes,
        "longitude": longitudes,
    },
)

In [35]:
dataset = dataset.rio.write_crs("EPSG:4326")
dataset.to_netcdf('test.nc')

In [4]:
scn =Scene(filenames=file_names,reader='modis_l1b')

In [5]:
scn.available_dataset_names()

['1',
 '10',
 '11',
 '12',
 '13hi',
 '13lo',
 '14hi',
 '14lo',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'latitude',
 'longitude',
 'satellite_azimuth_angle',
 'satellite_zenith_angle',
 'solar_azimuth_angle',
 'solar_zenith_angle']

In [48]:
scn.load(['1'])
scn['1'].values.shape

(2030, 1354)

In [8]:
scn['1'].attrs.keys()

dict_keys(['name', 'resolution', 'calibration', 'coordinates', 'wavelength', 'file_type', 'modifiers', 'units', 'standard_name', 'platform_name', 'sensor', 'rows_per_scan', 'start_time', 'end_time', 'reader', 'area', '_satpy_id', 'ancillary_variables'])

In [45]:
scn['1'].resolution

1000

In [36]:
scn.load(['latitude', 'longitude'])
lons5km = scn['longitude'].values
lats5km = scn['latitude'].values

In [ ]:
dataset = xr.Dataset(
    {
        "reflectance": (["latitude", "longitude"], scn['1'].values),
    },
    coords={
        "longitude": (["latitude", "longitude"], lons5km),
        "latitude": (["latitude", "longitude"], lats5km),
    },
)

In [50]:
dataset = dataset.rio.write_crs("EPSG:4326")
dataset.to_netcdf('test.nc')

In [29]:
data = xr.Dataset(
    {
        "latitude": (["x", "y"], lats5km),
        "longitude": (["x", "y"], lons5km),
    },
    coords={
        "x": np.arange(nx),
        "y": np.arange(ny),
    },
)

array([[ 125.32822 ,  125.35336 ,  125.378426, ..., -119.27483 ,
        -118.801765, -118.330765],
       [ 125.272995,  125.29819 ,  125.32331 , ..., -119.13418 ,
        -118.65962 , -118.187195],
       [ 125.21774 ,  125.243   ,  125.26817 , ..., -118.99242 ,
        -118.51639 , -118.04253 ],
       ...,
       [  75.86124 ,   75.79294 ,   75.72496 , ...,   11.934463,
          11.757427,   11.579187],
       [  75.82467 ,   75.75647 ,   75.68858 , ...,   11.945921,
          11.769108,   11.591087],
       [  75.788155,   75.72005 ,   75.65226 , ...,   11.957309,
          11.780708,   11.602909]], dtype=float32)

In [3]:
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
import os
from pyhdf.SD import SD, SDC
import numpy as np
import pandas as pd
import re
from pyproj import Transformer
import xarray as xr
#from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

In [21]:
test = xr.open_dataset(file_names[0], engine='netcdf4')

In [23]:
def mod_y_x(hdf):

    gridmeta = hdf.attributes(full=1)["StructMetadata.0"][0]

    ul_regex = re.compile(r'''UpperLeftPointMtrs=
''', re.VERBOSE)

    match = ul_regex.search(gridmeta)
    print(match)
    x0 = float(match.group('upper_left_x'))
    y0 = float(match.group('upper_left_y'))

    lr_regex = re.compile(r'''LowerRightMtrs=
''', re.VERBOSE)
    match = lr_regex.search(gridmeta)
    x1 = float(match.group('lower_right_x'))
    y1 = float(match.group('lower_right_y'))

    nx, ny = data[0].shape
    x = np.linspace(x0, x1, nx*2+1)
    y = np.linspace(y0, y1, ny*2+1)

    x = x[1::2]
    y = y[1::2]
    return y, x

In [24]:
dss = []

for file in file_names:
  hdf = SD(file, SDC.READ)
  DATAFIELD_NAME = 'EV_1KM_RefSB'

  # get the dataset fill value
  _FillValue = hdf.select(DATAFIELD_NAME).getfillvalue()

  #get the dataset min and max values: valid_range
  range = hdf.select(DATAFIELD_NAME).getrange()

  # get the dataset calibration coefficients scale_factor, scale_factor_err, add_offset, add_offset_err, calibrated_nt
  #cal_coef = hdf.select(DATAFIELD_NAME).getcal()

  # Read dataset
  data2D = hdf.select(DATAFIELD_NAME)
  data = data2D[:,:].astype('float32')
  #data = data.reshape(1, data.shape[0], data.shape[1])
  data = np.where((data==_FillValue) | (data < range[0]) | (data > range[1]),
                  np.nan, data)# * cal_coef[0]

  # get coordinates
  lat, lon = mod_y_x(hdf)

  da = xr.DataArray(
      data=data,
      dims=["time", "y", "x"],
      coords=dict(
          x=(["x"], lon),
          y=(["y"], lat),
          time=pd.date_range("2021-01-01", periods=1),

      ),

  )

  dss.append(da)

GROUP=SwathStructure
	GROUP=SWATH_1
		SwathName="MODIS_SWATH_Type_L1B"
		GROUP=Dimension
			OBJECT=Dimension_1
				DimensionName="Band_250M"
				Size=2
			END_OBJECT=Dimension_1
			OBJECT=Dimension_2
				DimensionName="Band_500M"
				Size=5
			END_OBJECT=Dimension_2
			OBJECT=Dimension_3
				DimensionName="Band_1KM_RefSB"
				Size=15
			END_OBJECT=Dimension_3
			OBJECT=Dimension_4
				DimensionName="Band_1KM_Emissive"
				Size=16
			END_OBJECT=Dimension_4
			OBJECT=Dimension_5
				DimensionName="10*nscans"
				Size=2030
			END_OBJECT=Dimension_5
			OBJECT=Dimension_6
				DimensionName="Max_EV_frames"
				Size=1354
			END_OBJECT=Dimension_6
			OBJECT=Dimension_7
				DimensionName="2*nscans"
				Size=406
			END_OBJECT=Dimension_7
			OBJECT=Dimension_8
				DimensionName="1KM_geo_dim"
				Size=271
			END_OBJECT=Dimension_8
		END_GROUP=Dimension
		GROUP=DimensionMap
			OBJECT=DimensionMap_1
				GeoDimension="2*nscans"
				DataDimension="10*nscans"
				Offset=2
				Increment=5
			END_OBJECT=

AttributeError: 'NoneType' object has no attribute 'group'

In [35]:
gridmeta = hdf.attributes(full=1)["StructMetadata.0"][0]

ul_regex = re.compile(r'''Upper Left=
''', re.VERBOSE)

match = ul_regex.search(gridmeta)
print(match)
x0 = float(match.group('Upper_Left'))
y0 = float(match.group('Upper_Left'))


None


AttributeError: 'NoneType' object has no attribute 'group'

In [38]:
import json
print(json.dumps(gridmeta, indent=4))

"GROUP=SwathStructure\n\tGROUP=SWATH_1\n\t\tSwathName=\"MODIS_SWATH_Type_L1B\"\n\t\tGROUP=Dimension\n\t\t\tOBJECT=Dimension_1\n\t\t\t\tDimensionName=\"Band_250M\"\n\t\t\t\tSize=2\n\t\t\tEND_OBJECT=Dimension_1\n\t\t\tOBJECT=Dimension_2\n\t\t\t\tDimensionName=\"Band_500M\"\n\t\t\t\tSize=5\n\t\t\tEND_OBJECT=Dimension_2\n\t\t\tOBJECT=Dimension_3\n\t\t\t\tDimensionName=\"Band_1KM_RefSB\"\n\t\t\t\tSize=15\n\t\t\tEND_OBJECT=Dimension_3\n\t\t\tOBJECT=Dimension_4\n\t\t\t\tDimensionName=\"Band_1KM_Emissive\"\n\t\t\t\tSize=16\n\t\t\tEND_OBJECT=Dimension_4\n\t\t\tOBJECT=Dimension_5\n\t\t\t\tDimensionName=\"10*nscans\"\n\t\t\t\tSize=2030\n\t\t\tEND_OBJECT=Dimension_5\n\t\t\tOBJECT=Dimension_6\n\t\t\t\tDimensionName=\"Max_EV_frames\"\n\t\t\t\tSize=1354\n\t\t\tEND_OBJECT=Dimension_6\n\t\t\tOBJECT=Dimension_7\n\t\t\t\tDimensionName=\"2*nscans\"\n\t\t\t\tSize=406\n\t\t\tEND_OBJECT=Dimension_7\n\t\t\tOBJECT=Dimension_8\n\t\t\t\tDimensionName=\"1KM_geo_dim\"\n\t\t\t\tSize=271\n\t\t\tEND_OBJECT=Dimension_

In [20]:
xv, yv = np.meshgrid(test.Latitude, test.Longitude)
transformer = Transformer.from_crs("+proj=sinu +R=6371007.181 +nadgrids=@null +wktext",
                                   "+init=EPSG:4326")

x, y= transformer.transform(xv, yv)

NameError: name 'test' is not defined